In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import hashlib
import json
import networkx as nx
import os

In [2]:
cvefixes = pd.read_csv("CVEfixes/CVEfixes_new.csv")
msr = pd.read_csv("Bigvul/MSR.csv").drop('Unnamed: 0', axis=1)
combined_df = pd.read_csv("../datasets_/combined_dataset.csv").drop('Unnamed: 0', axis=1)

# Load the JSON file representing the hierarchical structure
with open("preprocessed_datasets/debug_datasets/graph_all_paths.json", "r") as json_file:
    cwe_paths = json.load(json_file)

In [3]:
print(cvefixes.isna().sum())
cvefixes

code      20
cwe_id    20
cve_id    20
vul       20
dtype: int64


,code,cwe_id,cve_id,vul
0,expand_dynamic_string_token (struct link_map *...,252.0,CVE-1999-0199,1.0
1,"_dl_dst_substitute (struct link_map *l, const ...",252.0,CVE-1999-0199,1.0
2,"_dl_dst_count (const char *name, int is_path)\...",252.0,CVE-1999-0199,1.0
3,"vsyslog(pri, fmt, ap)\n\tint pri;\n\tregister ...",252.0,CVE-1999-0199,1.0
4,init_syntax_once ()\n{\n register int c;\n ...,252.0,CVE-1999-0199,1.0
...,...,...,...,...
69204,const delimiter = `ghadelimiter_${uuidv4()...,0.0,CVE-2022-35954,0.0
69205,private boolean isFileWithinDirectory(\n ...,0.0,CVE-2022-36007,0.0
69206,func TestNegativePowerLevels(t *testing.T) {\n...,0.0,CVE-2022-36009,0.0
69207,func NewPowerLevelContentFromEvent(event *Even...,0.0,CVE-2022-36009,0.0


In [4]:
print(msr.isna().sum())
msr

code          0
cwe_id        0
cve_id    12638
vul           0
dtype: int64


,code,cwe_id,cve_id,vul
0,static PHP_FUNCTION(preg_match)\n{\n\tphp_do_p...,119,CVE-2015-8382,0
1,static PHP_FUNCTION(preg_match_all)\n{\n\tphp_...,119,CVE-2015-8382,0
2,static PHP_FUNCTION(preg_replace)\n{\n\tpreg_r...,119,CVE-2015-8382,0
3,static PHP_FUNCTION(preg_replace_callback)\n{\...,119,CVE-2015-8382,0
4,static PHP_FUNCTION(preg_filter)\n{\n\tpreg_re...,119,CVE-2015-8382,0
...,...,...,...,...
145738,void impeg2d_dec_p_mb_params(dec_state_t *ps_d...,119,CVE-2016-0835,1
145739,void impeg2d_dec_pnb_mb_params(dec_state_t *ps...,119,CVE-2016-0835,1
145740,int equalizer_get_parameter(effect_context_t *...,200,CVE-2017-0402,1
145741,"uint8_t rfc_parse_data(tRFC_MCB* p_mcb, MX_FRA...",125,CVE-2018-9503,1


In [5]:
combined_df.head(5)

,code,cwe_id,cve_id,vul,count,assignedclass
0,"process_chpw_request(krb5_context context, voi...",20,CVE-2002-2443,1,23049,20
1,"init_ctx_reselect(OM_uint32 *minor_status, spn...",415,CVE-2014-4343,1,1111,666
2,"acc_ctx_cont(OM_uint32 *minstat,\n\t gss_b...",476,CVE-2014-4344,1,4973,703
3,krb5_ldap_get_password_policy_from_dn(krb5_con...,476,CVE-2014-5353,1,4973,703
4,check_rpcsec_auth(struct svc_req *rqstp)\n{\n ...,284,CVE-2014-9422,1,2467,284


In [6]:
len(cwe_paths),cwe_paths

(171,
 {'0': ['10000-0'],
  '20': ['10000-1000-707-20'],
  '22': ['10000-1000-664-706-22'],
  '23': ['10000-1000-664-706-22-23'],
  '27': ['10000-1000-664-706-22-23-27'],
  '59': ['10000-1000-664-706-59'],
  '73': ['10000-1000-664-669-642-73', '10000-1000-664-610-73'],
  '74': ['10000-1000-707-74'],
  '77': ['10000-1000-707-74-77'],
  '78': ['10000-1000-707-74-77-78'],
  '79': ['10000-1000-707-74-79'],
  '80': ['10000-1000-707-74-79-80'],
  '88': ['10000-1000-707-74-77-88'],
  '89': ['10000-1000-707-74-943-89'],
  '90': ['10000-1000-707-74-943-90'],
  '91': ['10000-1000-707-74-91'],
  '93': ['10000-1000-707-74-93'],
  '94': ['10000-1000-707-74-94', '10000-1000-664-913-94'],
  '98': ['10000-1000-664-706-98', '10000-1000-664-669-829-98'],
  '113': ['10000-1000-707-74-93-113', '10000-1000-435-436-113'],
  '116': ['10000-1000-707-116'],
  '119': ['10000-1000-664-118-119'],
  '120': ['10000-1000-664-118-119-120'],
  '121': ['10000-1000-664-118-119-787-121', '10000-1000-664-118-119-788-121']

In [7]:
filtered_df = cvefixes[cvefixes.isna().any(axis=1)]
print(filtered_df.shape)
filtered_df.isna().sum() # 20 nan in code, 20 nan in all other cols


(40, 4)


code      20
cwe_id    20
cve_id    20
vul       20
dtype: int64

In [8]:
cvefixes = cvefixes.dropna()
print(cvefixes.isna().sum())
cvefixes['cwe_id'] = cvefixes['cwe_id'].astype(int)
cvefixes['vul'] = cvefixes['vul'].astype(int)
cvefixes.head(5)

code      0
cwe_id    0
cve_id    0
vul       0
dtype: int64


/tmp/ipykernel_4803/436753690.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cvefixes['cwe_id'] = cvefixes['cwe_id'].astype(int)
/tmp/ipykernel_4803/436753690.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cvefixes['vul'] = cvefixes['vul'].astype(int)


,code,cwe_id,cve_id,vul
0,expand_dynamic_string_token (struct link_map *...,252,CVE-1999-0199,1
1,"_dl_dst_substitute (struct link_map *l, const ...",252,CVE-1999-0199,1
2,"_dl_dst_count (const char *name, int is_path)\...",252,CVE-1999-0199,1
3,"vsyslog(pri, fmt, ap)\n\tint pri;\n\tregister ...",252,CVE-1999-0199,1
4,init_syntax_once ()\n{\n register int c;\n ...,252,CVE-1999-0199,1


In [9]:
msr = msr.dropna()
msr.drop(columns=['Unnamed: 0'], inplace=True) 
msr.head(5)

KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
# Add hash column for code, to check for uniqueness
cvefixes['code.digest'] = cvefixes['code'].apply(lambda x: hashlib.sha1(x.encode()).hexdigest())
msr['code.digest'] = msr['code'].apply(lambda x: hashlib.sha1(x.encode()).hexdigest())

# # Remove code column
# cvefixes.drop(columns=['code'], inplace=True)
# msr.drop(columns=['code'], inplace=True)

# Combine datasets
alltbl = pd.concat([cvefixes, msr]).drop_duplicates()
print(alltbl.nunique())
alltbl['classpath'] = alltbl['cwe_id'].astype(str).map(cwe_paths)
alltbl['cwe_id'] = alltbl['cwe_id'].astype(int)
alltbl

In [ ]:

# statistics about codes in the same cve_id that are vulnerable and have different cwe_ids assigned
grouped = alltbl[alltbl['cwe_id'] != 0].groupby(['cve_id', 'code.digest'])['cwe_id'].agg([('cwe_id_count', 'nunique'), ('cwe_ids', lambda x: sorted(list(x.unique())))])

filtered = grouped[grouped['cwe_id_count'] > 1].reset_index()

print(filtered['cwe_ids'].value_counts().to_string())

In [ ]:
# Create a DataFrame with counts for each cwe_id
cwecounts = alltbl.groupby('cwe_id').size().reset_index(name='count')
print(cwecounts.to_string())

In [ ]:
def create_graph_from_json(paths_dict_data, max_depth=None):
    
    G = nx.DiGraph()

    def add_path_to_graph(path):
        nodes = list(map(int, path.split('-')))
        if max_depth:
            max_level = min(max_depth, len(nodes) - 1)
            for i in range(max_level):
                G.add_edge(nodes[i], nodes[i+1])
        else:
            for i in range(len(nodes) - 1):
                G.add_edge(nodes[i], nodes[i+1])

    # Add edges from the paths in the JSON data
    for key, paths_list in paths_dict_data.items():
        for path in paths_list:
            add_path_to_graph(path)
            
    return G

graph = create_graph_from_json(cwe_paths)
node_list = list(graph.nodes)

In [ ]:
# for reassignment, we are going to use a tree.
# if a node has multiple predecessors, we choose the predecessor with more items in the dataset.

def choose_favorite_parent(node):
    parents = list(graph.predecessors(node))
    if len(parents) == 1:
        return parents[0]
    num_parent_samples = list()
    for p in parents:
        parent_family = nx.ancestors(graph, p) - set.union(*(nx.ancestors(graph, otherp) for otherp in parents if otherp != p))
        if parent_family:
            parent_descendants = set.union(*(nx.descendants(graph, x) for x in parent_family)).union(parent_family)
        else:
            parent_descendants = parent_family
        num_parent_samples.append(
            cwecounts[cwecounts['cwe_id'].isin(parent_descendants)]['count'].sum()
        )
    return(parents[num_parent_samples.index(max(num_parent_samples))])

graph_unique = nx.DiGraph()
for node in list(nx.topological_sort(graph))[1:]:
    graph_unique.add_edge(choose_favorite_parent(node), node)

In [ ]:
def splitclass(df, split_class):
    df = df.copy()
    results = [df.loc[df['assignedclass'] != split_class]]  # items that are not touched
    split_into = set(graph_unique.neighbors(split_class))
    for cwe_id in df.loc[df['assignedclass'] == split_class]['cwe_id']:
        new_assignedclasses = split_into.intersection(
            nx.ancestors(graph_unique, cwe_id).union({cwe_id})
        )
        if not new_assignedclasses:
            # no children to split into -> don't split
            new_assignedclasses = {split_class}
        for ac in new_assignedclasses: # could be multiple
            newline = df.loc[(df['cwe_id'] == cwe_id) & (df['assignedclass'] == split_class)].copy()
            newline['assignedclass'] = ac
            results.append(newline)
    return pd.concat(results).drop_duplicates()

In [ ]:
# cweiter = cwecounts.copy()
# cweiter['assignedclass'] = 10000
# cweiter = cweiter[:7]


In [ ]:
# print(cweiter)

In [ ]:
# cweiter = splitclass(cweiter, 706)
# print(cweiter)

In [ ]:
# cutoff = 1000
# get_worth_splitting(cweiter,cutoff)

In [ ]:
def check_worth_splitting(df, split_class, cutoff):
    num_above_cutoff = sum(df.loc[df['assignedclass'] == split_class]['count'] >= cutoff)
    return num_above_cutoff > 1

In [ ]:
def get_worth_splitting(df, cutoff):
    return [x for x in set(df['assignedclass']) if check_worth_splitting(df, x, cutoff)]

In [ ]:
cutoff = 1000 #################################################

In [ ]:
cweiter = cwecounts.copy()
cweiter['assignedclass'] = 10000
while True:
    splitting = get_worth_splitting(cweiter, cutoff)
    if not splitting:
        break
    cweiter = splitclass(cweiter, splitting[0])

manual_classes = [284]
for manual_cwe in manual_classes:
    cweiter = splitclass(cweiter, manual_cwe)

In [ ]:
cweiter

In [ ]:
len(cweiter['cwe_id'].unique())

In [ ]:
len(cweiter)

In [ ]:
check = cweiter.copy()
check = check[check['assignedclass']==284]

In [ ]:
print(splitclass(check, 284).to_string())

In [ ]:
check2 = cweiter.copy()
check2 = check2[check2['assignedclass']==404]
print(splitclass(check2, 404).to_string())

In [ ]:
cweiter.sort_values('assignedclass')

In [ ]:
print(cweiter.sort_values('assignedclass').to_string())
# 284, 404

In [ ]:
print(cweiter.groupby('assignedclass')['count'].sum().sort_values(ascending = False).reset_index().to_string())

In [ ]:
cweiter_stat = cweiter.groupby('assignedclass')['count'].sum().sort_values(ascending = False).reset_index()
cweiter_stat

In [ ]:
cweiter_stat['count'].sum()

In [ ]:
cweiter_stat[cweiter_stat['count']>=cutoff]['count'].sum() / cweiter_stat['count'].sum()

In [ ]:
cweiter_1000 = cweiter[cweiter['count']>=cutoff]
cweiter_1000.to_csv('preprocessed_datasets/assigined_cwe_cutoff1000.csv')

cweiter_1000

In [ ]:
cweiter

In [ ]:
# Merge or join the DataFrames based on cwe_id
def merge_assignclass_and_drop_by_cutoff(df,cweiter,cutoff,output_dir):
    df_reassign = df.merge(cweiter, on='cwe_id', how='left')
    df_reassign = df_reassign.drop('code.digest', axis=1)
    
    df_reassign = df_reassign[df_reassign['count'] >= cutoff]
    
    df_reassign.to_csv(output_dir)
    return df_reassign
    

In [ ]:
CVEfixes_reassign = merge_assignclass_and_drop_by_cutoff(cvefixes,cweiter,cutoff,output_dir='CVEfixes/CVEfixes_reassign.csv')
CVEfixes_reassign

In [ ]:
MSR_reassign = merge_assignclass_and_drop_by_cutoff(msr,cweiter,cutoff,output_dir='Bigvul/MSR_reassign.csv')
MSR_reassign

In [ ]:
# Combine two DataFrames and drop duplicates
combined_df = pd.concat([CVEfixes_reassign, MSR_reassign])
print(combined_df.shape)
combined_df = combined_df.drop_duplicates()
print(combined_df.shape)
# Display rows with NaN values
nan_rows = combined_df[combined_df.isna().any(axis=1)]
print(nan_rows)
combined_df = combined_df.dropna()
print(combined_df.shape)
combined_df

In [ ]:
data_path = '../datasets_/final_dataset'
combined_df.to_csv(f"{data_path}/combined_dataset.csv")

In [ ]:
unique_values = combined_df.nunique()
unique_values

In [ ]:
combined_df[combined_df['cwe_id']!=combined_df['assignedclass']]

In [ ]:
unique_cwe_ids = combined_df['assignedclass'].unique().tolist()
unique_cwe_ids = sorted(unique_cwe_ids)
print(len(unique_cwe_ids),unique_cwe_ids)

In [ ]:
# Count unique values in 'cwe_id' column
combined_before_reassign_df = pd.concat([cvefixes, msr])
combined_before_reassign_df = combined_before_reassign_df.drop_duplicates()
nan_rows = combined_before_reassign_df[combined_before_reassign_df.isna().any(axis=1)]
combined_before_reassign_df = combined_before_reassign_df.dropna()
print(combined_before_reassign_df.shape)

cwe_counts = combined_before_reassign_df['cwe_id'].value_counts()
# Plotting
plt.figure(figsize=(30, 6)) 
cwe_counts.plot(kind='bar', logy=True)
plt.xlabel('CWE ID')
plt.xticks(rotation=45)  # Rotate labels by 45 degrees
plt.ylabel('Counts')
plt.title('Counts of CWE IDs before Reassignment')
plt.show()

In [ ]:
# Count unique values in 'assignedclass' column
cwe_counts = combined_df['assignedclass'].value_counts()
# Plotting
cwe_counts.plot(kind='bar', logy=True)
plt.xlabel('Assignedclass')
plt.ylabel('Counts')
plt.title('Counts of CWE IDs after Reassignment')
plt.show()

## Stratified split dataset to train/val/test set by CVE id group

In [ ]:
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

# Function to plot the distribution
def plot_distribution(dataset, title, col_name='assignedclass'):
    plt.figure(figsize=(12, 6))
    col_counts = dataset[col_name].value_counts()
    sns.barplot(x=col_counts.index, y=col_counts.values)
    plt.title(title)
    plt.ylabel('Number of Occurrences')
    plt.xlabel(col_name)
    plt.xticks(rotation=90)  # Rotates the x-axis labels to be more readable
    plt.show()

def stratify_dataset(df, col_name='assignedclass'):
    initial_allocation = defaultdict(lambda: {"train": 0, "validation": 0, "test": 0})

    # Initialize datasets
    train, validation, test = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    random_state = 42


    for cwe_id, group in df.groupby(col_name):
    #     print("cwe_id, group",cwe_id, group)
        # For each cwe_id, allocate the first few cve_id groups to different datasets
        for i, (cve_id, cve_group) in enumerate(group.groupby('cve_id')):
            if i % 15 > 1:
                train = pd.concat([train, cve_group])
                initial_allocation[cwe_id]["train"] += len(cve_group)
            elif i % 15 == 1:
                validation = pd.concat([validation, cve_group])
                initial_allocation[cwe_id]["validation"] += len(cve_group)
            else:
                test = pd.concat([test, cve_group])
                initial_allocation[cwe_id]["test"] += len(cve_group)
    remaining_df = df[~df['cve_id'].isin(pd.concat([train, validation, test])['cve_id'])]
    print(remaining_df.shape)
    # if not remaining_df.empty:
    #     remaining_train, remaining_validation, remaining_test = split_group(remaining_df, len(remaining_df))
    #     train = pd.concat([train, remaining_train])
    #     validation = pd.concat([validation, remaining_validation])
    #     test = pd.concat([test, remaining_test])

    
    train = train.sample(frac=1, random_state=random_state).reset_index(drop=True)
    validation = validation.sample(frac=1, random_state=random_state).reset_index(drop=True)
    test = test.sample(frac=1, random_state=random_state).reset_index(drop=True)
    print(train.shape[0], validation.shape[0], test.shape[0])
    return train, validation, test


In [ ]:
# Split Reassigned Dataset
train, validation, test = stratify_dataset(combined_df, col_name='assignedclass')

train.to_csv(f'{data_path}/new_train_dataset.csv', index=False)
validation.to_csv(f'{data_path}/new_validation_dataset.csv', index=False)
test.to_csv(f'{data_path}/new_test_dataset.csv', index=False)

# Plotting for each dataset
plot_distribution(train, 'Training Set CWE ID Distribution', 'assignedclass')
plot_distribution(validation, 'Validation Set CWE ID Distribution','assignedclass')
plot_distribution(test, 'Test Set CWE ID Distribution','assignedclass')


In [ ]:
def guess_samples_per_cwe_id(validation, target_sample_size, col_name = 'assignedclass'):
    num_unique_cwe_ids = validation[col_name].nunique()

    samples_per_cwe_id = target_sample_size // num_unique_cwe_ids
    while samples_per_cwe_id < len(validation):
        total_samples = 0
        for cwe_id in validation[col_name].unique():
            samples_to_take = min(samples_per_cwe_id, len(validation[validation[col_name] == cwe_id]))
            total_samples += samples_to_take

        if total_samples >= target_sample_size:
            break
        samples_per_cwe_id += 1
    return samples_per_cwe_id

def create_balanced_validation_dataset(validation, target_sample_size, col_name='assignedclass',random_state=42):

    # Calculate how many samples to take per CWE ID
#     samples_per_cwe_id = guess_samples_per_cwe_id(validation, target_sample_size, col_name)
    num_unique_cwe_ids=21
    samples_per_cwe_id = target_sample_size // num_unique_cwe_ids
    
    # Sample the rows
    balanced_validation = pd.DataFrame()

    for cwe_id in validation[col_name].unique():
        
        # If there are not enough samples for a particular cwe_id, take all available
        samples_to_take = min(samples_per_cwe_id, len(validation[validation[col_name] == cwe_id]))
        sampled_group = validation[validation[col_name] == cwe_id].sample(n=samples_to_take, random_state=random_state)
        balanced_validation = pd.concat([balanced_validation, sampled_group])

    # Check if we have reached the target sample size
    current_sample_size = len(balanced_validation)

    # If the current sample size is less than the target, fill in the remaining with random samples from the validation set
    # IF it happens it would be a problem.
    if current_sample_size < target_sample_size:
        additional_samples = validation.sample(n=target_sample_size - current_sample_size, random_state=random_state)
        balanced_validation = pd.concat([balanced_validation, additional_samples])

    # Shuffle the balanced validation set
    balanced_validation = balanced_validation.sample(frac=1, random_state=random_state).reset_index(drop=True)

    # Update the validation DataFrame
    validation = balanced_validation
    print(validation.shape)
    return validation

In [ ]:
# Desired total sample size
target_sample_size = 5000
balanced_validation = create_balanced_validation_dataset(validation, target_sample_size, col_name='assignedclass')
# Output the sampled DataFrame
balanced_validation.to_csv(f'{data_path}/balanced_validation_dataset.csv', index=False)
plot_distribution(balanced_validation, 'Balanced Validation Set CWE ID Distribution')

# # Desired total sample size
# target_sample_size = 5000
# ori_balanced_validation = create_balanced_validation_dataset(ori_validation,target_sample_size, col_name='assignedclass')
# # Output the sampled DataFrame
# ori_balanced_validation.to_csv(f'{data_path}/ori_balanced_validation_dataset.csv', index=False)
# plot_distribution(ori_balanced_validation, 'Original Balanced Validation Set CWE ID Distribution')

# train.shape[0], validation.shape[0], test.shape[0]